In [ ]:
import numpy as np
from numpy import linalg as LA

from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing

from scipy.spatial.distance import cdist
from scipy.linalg import eigh


### Implementering kernels and do tests

In [ ]:
def rbf_kernel(X, sigma, diag=1):
    """"Basic SVM with predefined kernel matrix"""
    
    N = X.shape[0]
    K = np.zeros((N,N))

    for i in range(N):
        for j in range(N):
            if i == j:
                K[i,j] = diag
            else:
                x_i = X[i, :].reshape(1,-1)
                x_j = X[j, :].reshape(1,-1)

                K[i, j] = np.exp(-cdist(x_i, x_j, 'sqeuclidean') / (sigma ** 2)) # euclidean?
    return K

def make_D_matrix(K):
    K_sum = np.sum(K, axis=1)
    D = np.diag(K_sum)

    return D

def make_L_matrix(K, D):
    D_temp = np.diag( np.diag(D) ** -0.5 )
    L = D_temp @ K @ D_temp
    
    w, v = LA.eig(L) # w = eigenvalues, v = normalized (unit “length”) eigenvectors
    
    return L

def step_transfer(L, k=2):
    w, v = eigh(L)
    lambda_cut = w[-k]
    
    w = np.where(w >= lambda_cut, 1, 0)
    L_hat = np.dot(v, np.dot(np.diag(w), v.T))
    D_hat = np.diag(1/np.diag(L_hat))
    K_hat = D_hat**(1/2) @ L_hat @ D_hat**(1/2)
    
    return L_hat, D_hat, K_hat

def linear_step_transfer(L, k=2):
    w, v = eigh(L)
    lambda_cut = w[-k]
    w = np.where(w >= lambda_cut, w, 0)
    
    L_hat = np.dot(v, np.dot(np.diag(w), v.T))
    D_hat = np.diag(1/np.diag(L_hat))
    K_hat = D_hat**(1/2) @ L_hat @ D_hat**(1/2)

    return L_hat, D_hat, K_hat

def polynomial_transfer(L, D, K, t):
    L_hat = L ** t
    D_hat = np.diag(1/np.diag(L_hat))
    K_hat = D_hat**(1/2) @ D**(1/2) @ (LA.inv(D) @ K)**t @ D**(1/2) @ D_hat**(1/2)
    K_hat = preprocessing.scale(K_hat)

    return L_hat, D_hat, K_hat

def apply_transfer_func(L, D, K, hyperparams, type="linear"):
    """hyperparams: k for step and linear_step, t for polynomial"""
    if type == "linear":
        return L, D, K
    if type == "step":
        k = hyperparams['k']
        return step_transfer(L, k)
    if type == "linear_step":
        k = hyperparams['k']
        return linear_step_transfer(L)
    if type == "polynomial":
        t = hyperparams['t']
        return polynomial_transfer(L, D, K, t)
        
    raise ValueError("wrong argument")

def accuracy(t, y):
    val = 0.0
    N = len(t)
    for i in range(N):
        if t[i] == y[i]:
            val += 1
    
    return val / N

In [ ]:
def test_svm(X, Y, tf_fun, C=100, sigma=1, **kwargs):
    """Test SVM one time"""
    
    # Shuffle data
    np.random.seed(40)
    n_sample = len(X)
    order = np.random.permutation(n_sample)
    X = X[order]
    Y = Y[order].astype(np.float)
    
    # Make Kernel
    K = rbf_kernel(X, sigma)
    D = make_D_matrix(K)
    L = make_L_matrix(K, D)
    
    L, D, K = apply_transfer_func(L, D, K, kwargs, tf_fun)
    
    # Remove data without labels
    K_train = K[:70,:70]
    Y_train = Y[:70]
    
    K_test = K[70:100,:70]
    
    # Apply to SVM
    clf = SVC(kernel="precomputed", C=C)
    clf.fit(K_train, Y_train)
    
    y_pred = clf.predict(K_test)
    print("accuracy:", accuracy(y_pred, Y[70:100]))

def run_test_svm():
    iris = load_iris()
    X = iris.data[:,:2]
    y = iris.target

    X = X[y != 0]
    y = y[y != 0]
    
    test_svm(X, y, "linear")

run_test_svm()

In [ ]:
def validate_hyperparameters(X_train, y_train, C, sigma):
    """Validates hyperparamters using k-fold cross validation with k=10"""
    errors = []

    # used for indexing in loop
    fold_size = int(X_train.shape[0] / 10)

    for fold_n in range(10):
        # splits training data into 3 separate arrays
        x_splits = np.vsplit(X_train, [fold_n * fold_size, fold_n * fold_size + fold_size])

        # middle set is current validation set
        x_validation_set = x_splits[1]
        # merge first and second array from split to get training set
        x_training_set = np.vstack((x_splits[0], x_splits[2]))

        # do same thing for y labels
        y_splits = np.split(y_train, [fold_n * fold_size, fold_n * fold_size + fold_size])
        y_validation_set = y_splits[1]
        y_training_set = np.append(y_splits[0], y_splits[2])

        # get error for current fold
        errors.append(
            get_svm_error(
                x_training_set,
                x_validation_set,
                y_training_set,
                y_validation_set,
                C,
                sigma,
            )
        )
        
    errors = np.array(errors)   
    return errors.mean()


def get_svm_error(x_training_set, x_validation_set, y_training_set, y_validation_set, C, sigma):    
    N_train = x_training_set.shape[0]
    X_t = np.concatenate([x_training_set, x_validation_set])

    K = rbf_kernel(X_t, sigma)
    D = make_D_matrix(K)
    L = make_L_matrix(K, D)
    
    L, D, K = apply_transfer_func(L, D, K, {}, "linear")
    
    K_train = K[:N_train, :N_train]
    K_val = K[N_train:, :N_train]
    
    clf = SVC(kernel="precomputed", C=C)
    clf.fit(K_train, y_training_set)
    
    y_pred = clf.predict(K_val)
    
    err = 1 - accuracy(y_pred, y_validation_set)
    return err


def find_hyperparameters(X_train, y_train):
    #c_values = np.arange(0.1, 10, 0.2)
    #sigma_values = np.arange(0.1, 2, 0.2)
    c_values = [0.01, 0.1, 1, 10, 100, 1000]
    sigma_values = [0.01, 0.1, 1, 10, 100, 1000]
    
    lowest_error = 1.0
    best_parameter_values = [0, 0]
    
    iters = len(c_values) * len(sigma_values)
    i = 0

    for c in c_values:
        for sigma in sigma_values:
            current_error = validate_hyperparameters(X_train, y_train, c, sigma)

            # print("c: {}\t sigma: {}\t error: {}".format(c, sigma, current_error))
            if (current_error < lowest_error):
                lowest_error = current_error
                best_parameter_values = c, sigma
            
            if i % 4 == 0:
                print("{} / {}".format(i, iters))
            i += 1
      
    res = "c: {}\t sigma: {}\t error: {}".format(best_parameter_values[0], best_parameter_values[1], lowest_error)
    return res


def run_find_hyperparameters():
    iris = load_iris()
    X = iris.data[:,:2]
    y = iris.target

    X = X[y != 0]
    y = y[y != 0]
    
    np.random.seed(40)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    res = find_hyperparameters(X_train, y_train)
    print(res)

run_find_hyperparameters()


## Experiments
Here under we can start to run experiment with the hyperparams we found above